# Finite State Machines 

Overview

1. Fundamentals
2. Turnstile
3. Traffic Jam
4. Sudoku

But first we must be ready:

In [15]:
import sys
assert (sys.version_info.major, sys.version_info.minor) >= (3,7)
%pip install graph-theory --upgrade --no-cache -q
from graph import Graph

Note: you may need to restart the kernel to use updated packages.


## Fundamentals

Nomenclature

- A finite state machine is a mathematical model of computation. 
- The model is in exactly one state at any time.
- The model can change from one state to another using predetermined paths.
- All possible states is called the "solution landscape"



# Turnstile

![turnstile](images/turnstile.png)

In [16]:
from graph.finite_state_machine import FiniteStateMachine

locked, unlocked = 'locked', 'unlocked'  # states
push, coin = 'push', 'coin'  # actions

fsm = FiniteStateMachine()
fsm.add_transition(locked, coin, unlocked)  # turnstile is locked. Put in coin to unlock.
fsm.add_transition(unlocked, push, locked)  # turnstile is unlocked. Push the rotor to lock.
fsm.add_transition(locked, push, locked)  # turnstile is locked. Pushing does not unlock.
fsm.add_transition(unlocked, coin, unlocked)  # turnstile is unlocked. Adding more coins does not change state.

fsm.set_initial_state(locked)  # set initial state.

In [17]:
# check options: pay and go
set(fsm.options())

{'coin', 'push'}

In [18]:
fsm.current_state == locked

True

In [19]:
# now insert coin:
fsm.next(action=coin)

In [20]:
# check options:
set(fsm.options())

{'coin', 'push'}

In [21]:
fsm.current_state == unlocked

True

In [22]:
# ok it's unlocked - so we push:
fsm.next(action=push)
# and check if it locks behind us:
fsm.current_state == locked

True

# Traffic Jam

We have a small intersection like this:

![3-3 traffic jam](images/3-3-cart-traffic-jam_initial.png)

The solution we would like to see looks like this:

![3-3 traffic jan final](images/3-3-cart-traffic-jam.png)

And here's the solution:

![3-3 traffic jam solution](images/traffic_bi_directional.gif)

Q: How did I get to that?

A: 3 steps:

1. Define the system as a FSM.
2. Define the initial state
3. Search through the solution landscape until the state resembles the final state.

Let's start with the map:

![tjs-map](images/tjs-map.png)

In [23]:
g = Graph()
edges = [(1,2),(2,3),(2,4)]
for edge in edges:
    g.add_edge(*edge, bidirectional=True)

Next I add the load

![tjs with pug](images/tjs-map-pugs.png)

In [24]:
loads = [
    {'id':'red', 'start': 1, 'ends': 3}, 
    {'id': 'blue', 'start': 3, 'ends': 1}
]

In [25]:
from graph.traffic_scheduling_problem import jam_solver

In [26]:
solution = jam_solver(g,loads)

queue exhausted


In [27]:
solution

[{'red': (1, 2)},
 {'red': (2, 4), 'blue': (3, 2)},
 {'blue': (2, 1), 'red': (4, 2)},
 {'red': (2, 3)}]

In [28]:
# in plain english:
for move in solution:
    for color,(start,end) in move.items():
        print(f"{color} moves from {start} to {end}")

red moves from 1 to 2
red moves from 2 to 4
blue moves from 3 to 2
blue moves from 2 to 1
red moves from 4 to 2
red moves from 2 to 3


So what kind of state machine is the traffic jam solver using?

![tjs](images/3-3-tree-of-states.png)

Making a deep-copy of a state machine may be easy, but will quick consume a terrible amount of memory.

It's far better to use a graph.

In [29]:
state_0 = ((1, 'red'), (2, None), (3, 'blue'), (4, None))
state_1 = ((1, None), (2, 'red'), (3, 'blue'), (4, None))
state_2 = ((1, 'red'), (2, 'blue'), (3, None), (4, None))

fsm_graph = Graph()
fsm_graph.add_edge(state_0, state_1)
fsm_graph.add_edge(state_0, state_2)

The search now becomes "make a change", check if it exists in the graph and add it to the graph. However the state definition contains a lot of Nones that don't really add much.

Suggestion: Let's just drop that information.

New graph:

In [30]:
state_0 = ((1, 'red'), (3, 'blue'))
state_1 = ((2, 'red'), (3, 'blue'))
state_2 = ((1, 'red'), (2, 'blue'))

fsm_graph = Graph()
fsm_graph.add_edge(state_0, state_1)
fsm_graph.add_edge(state_0, state_2)

That's nicer to read.

Let's make a solver for that:

In [32]:
# we need the "map" from earlier:
the_map = Graph()
edges = [(1,2),(2,3),(2,4)]
for edge in edges:
    the_map.add_edge(*edge, bidirectional=True)

# we need the "frontier"
job_queue = [state_1, state_2]

# we need a "final state"
final_state = tuple(sorted([(3,'red'), (1,'blue')]))

# we need a search
while job_queue:
    state = job_queue.pop()
    occupied = {position for position,item in state}
    for position, item in state:  # 'red' on position 2.
        for option in the_map.nodes(from_node=position):  # positions {1,4}
            if option in occupied:
                continue  # skip it.
            
            new_state = tuple(sorted([(option, item)] + [(o,i) for o,i in state if i != item]))
            
            if new_state in fsm_graph:
                continue  # we've already seen it.
            
            # we add the new state to be explored.
            fsm_graph.add_edge(state, new_state)
            job_queue.append(new_state)

            if new_state == final_state:
                break

As the solver has made the finite state diagram for us (hurray!) we can have a look:

In [33]:
fsm_graph.nodes()

[((1, 'red'), (3, 'blue')),
 ((2, 'red'), (3, 'blue')),
 ((1, 'red'), (2, 'blue')),
 ((1, 'red'), (4, 'blue')),
 ((2, 'red'), (4, 'blue')),
 ((3, 'red'), (4, 'blue')),
 ((2, 'blue'), (3, 'red')),
 ((1, 'blue'), (3, 'red')),
 ((1, 'blue'), (2, 'red')),
 ((1, 'blue'), (4, 'red')),
 ((2, 'blue'), (4, 'red')),
 ((3, 'blue'), (4, 'red'))]

To find the fewest moves to resolve the traffic jam, we can just ask the finite state diagram for the shortest path from the `initial state` to the `final state`:

In [34]:
fsm_graph.shortest_path(state_0, final_state)

(6,
 [((1, 'red'), (3, 'blue')),
  ((1, 'red'), (2, 'blue')),
  ((1, 'red'), (4, 'blue')),
  ((2, 'red'), (4, 'blue')),
  ((3, 'red'), (4, 'blue')),
  ((2, 'blue'), (3, 'red')),
  ((1, 'blue'), (3, 'red'))])

And there you have it! The solution from earlier:

```
red moves from 1 to 2
red moves from 2 to 4
blue moves from 3 to 2
blue moves from 2 to 1
red moves from 4 to 2
red moves from 2 to 3
```

and the solution above shows, state by state what has changed. Note that the two valid solutions interchangeable. 

# Sudoku

Solving a sudoku is no different.

